#Parsing For a Demo HDFS dataset# 

In [13]:
import numpy as np
import pandas as pd
import re

import sys
from logparser import Drain

input_log_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\logs\HDFS_2k.log'


# data_list = []

# with open(input_file, 'r') as file:
#     for line in file:
#         columns = line.split()

#         if len(columns) >= 6:
#             data_list.append(columns[5:])

# file.close()

# for item in data_list:
#     print(item)


input_dir = "\logs"  # The input directory of log file
output_dir = "\parse"  # The output directory of parsing results
log_file_all = input_log_file # The input log file name
log_format = "<Date> <Time> <Pid> <Level> <Component>: <Content>"  # HDFS log format
# Regular expression list for optional preprocessing (default: [])
regex = [
    r"blk_(|-)[0-9]+",  # block id
    r"(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)",  # IP
    r"(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$",  # Numbers
]
st = 0.5  # Similarity threshold
depth = 4  # Depth of all leaf nodes

# run on training dataset
parser = Drain.LogParser(
    log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex
)
parser.parse(log_file_all)

structuredLog = open("HDFS_2k.log_structured.csv","r")
df = pd.read_csv(structuredLog)

print(df)

# selected_columns = df[['EventId', 'ParameterList']]

# # Combine 'EventId' and 'ParameterList' into a new column 'CombinedParameterList'
# selected_columns['CombinedParameterList'] = selected_columns.apply(lambda row: f"{row['EventId']}, {row['ParameterList']}", axis=1)

# # Save the result to a new CSV file
# selected_columns.to_csv('features.csv', index=False)





Parsing file: E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\logs\HDFS_2k.log
Total lines:  2000
Processed 50.0% of log lines.
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:01.446507]
      LineId   Date    Time    Pid Level                     Component  \
0          1  81109  203615    148  INFO  dfs.DataNode$PacketResponder   
1          2  81109  203807    222  INFO  dfs.DataNode$PacketResponder   
2          3  81109  204005     35  INFO              dfs.FSNamesystem   
3          4  81109  204015    308  INFO  dfs.DataNode$PacketResponder   
4          5  81109  204106    329  INFO  dfs.DataNode$PacketResponder   
...      ...    ...     ...    ...   ...                           ...   
1995    1996  81111  101621  24902  INFO      dfs.DataNode$DataXceiver   
1996    1997  81111  101735  26595  INFO  dfs.DataNode$PacketResponder   
1997    1998  81111  101804  26494  INFO      dfs.DataNode$DataXceiver   
1998    1999  81111  101954  26414  INF

#Grouping for demo HDFS datset#

In [11]:

#grouping for demo files
import csv

input_log_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\parse\demo_data\HDFS_2k.log_structured.csv'

def extract_data(input_file, output_file):
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Extract EventId and strings of the form "blk_*"
        data_list = [(row['EventId'], param) for row in reader for param in row['ParameterList'].strip("[]").replace("'", "").split(', ') if param.startswith('blk_')]

    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['EventId', 'BlockString']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for data in data_list:
            writer.writerow({'EventId': data[0], 'BlockString': data[1]})

extract_data(input_log_file, 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\ blk_eve_ids.csv')


In [13]:
#grouping part 2 for demo files

import csv
from collections import defaultdict

def extract_and_group_data(input_file, output_file):
    block_events = defaultdict(list)

    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Extract EventId and strings of the form "blk_*"
        for row in reader:
            event_id = row['EventId']
            for param in row['BlockString'].strip("[]").replace("'", "").split(', '):
                if param.startswith('blk_'):
                    block_id = param
                    block_events[block_id].append(event_id)

    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['BlockId', 'EventIds']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for block_id, event_ids in block_events.items():
            writer.writerow({'BlockId': block_id, 'EventIds': ', '.join(event_ids)})

extract_and_group_data('E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\demo\ blk_eve_ids.csv', 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\demo\grouped_output_file.csv')


#Grouping for actual project data# 

In [14]:
#grouping for proj data HDFS!

import csv
import re

def extract_event_and_block(content):
    # Define a regular expression to extract "blk_*" pattern
    block_pattern = r'blk_[\d]+'
    
    # Find all occurrences of "blk_*" in the content
    blocks = re.findall(block_pattern, content)
    
    return blocks

# Input and output file paths
input_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\parse\proj_data\HDFS_100k.log_structured.csv' 
output_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\ blk_eve.csv'

with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['EventId', 'Block']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for row in reader:
        event_id = row['EventId']
        content = row['Content']
        blocks = extract_event_and_block(content)
        
        for block in blocks:
            writer.writerow({'EventId': event_id, 'Block': block})


Extraction complete. Check the output file: E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\ blk_eve.csv


In [18]:
#grouping part 2 fro proj data HDFS
import pandas as pd

input_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\ blk_eve.csv'  # Update with your actual file path
output_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\sequence.csv'

df = pd.read_csv(input_file)

# Group by Block and aggregate unique EventIds as a set for each group
grouped_df = df.groupby('Block')['EventId'].agg(set).reset_index()

grouped_df.to_csv(output_file, index=False)

print("Grouping complete. Check the output file:", output_file)


Grouping complete. Check the output file: E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\sequence.csv


#labelling for hdfs project data#

In [21]:
import pandas as pd

extracted_info_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\sequence.csv'  # Update with your actual file path
label_info_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\HDFS.anomaly_label.csv'  # Update with your actual file path
output_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\labelled_sequnce.csv'

# Read the extracted info and label info into pandas DataFrames
extracted_df = pd.read_csv(extracted_info_file)
label_df = pd.read_csv(label_info_file)

merged_df = pd.merge(extracted_df, label_df, how='left', left_on='Block', right_on='BlockId')

merged_df = merged_df[['EventId', 'Label']]

merged_df.to_csv(output_file, index=False)

print("Merging complete. Check the output file:", output_file)


Merging complete. Check the output file: E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\labelled_sequnce.csv


#Grouping and labelling for BGL data#

In [3]:
#bgl SEQUNCE FILE GEENRATION
import pandas as pd

csv_file_path = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\parse\proj_data\BGL_4700k.log_structured.csv'

df = pd.read_csv(csv_file_path)

grouped_data = df.groupby('Node').agg({'EventId': lambda x: ', '.join(map(str, x)),
                                       'Label': lambda x: 'Normal' if all(label == '-' for label in x) else 'Anomalous'}).reset_index()

grouped_data.drop('Node', axis=1, inplace=True)

output_csv_path = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\BGL_sequnece.csv'
grouped_data.to_csv(output_csv_path, index=False)

print(f"Grouped data with labels (without Node column) saved to {output_csv_path}")



Grouped data with labels (without Node column) saved to E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\BGL_sequnece.csv


#Grouping and labelling for Spirit Dataset#

In [2]:
#grouping for SPIRIT dset!

import pandas as pd

df = pd.read_csv('E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\parse\proj_data\Spirit.log_structured.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

interval_size = 113

df['WindowIndex'] = (df['Timestamp'].astype('int64') // 10**9 // interval_size).astype('int64')

grouped_data = df.groupby('WindowIndex')

result_list = []

for group_name, group_data in grouped_data:
    event_ids = set(group_data['EventId'])
    label = 'Normal' if all(group_data['Label'] == '-') else 'Anomalous'
    result_list.append({'EventIds': event_ids, 'Label': label})

result_df = pd.DataFrame(result_list)

result_df.to_csv('E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\Spirit_sequence.csv', index=False)

#Grouping and labelling for Thunderbird Dataset#

In [ ]:
import pandas as pd

df = pd.read_csv('E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\parse\proj_data\Thunderbird.log_structured.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

interval_size = 900

df['WindowIndex'] = (df['Timestamp'].astype('int64') // 10**9 // interval_size).astype('int64')

grouped_data = df.groupby('WindowIndex')

result_list = []

for group_name, group_data in grouped_data:
    event_ids = set(group_data['EventId'])
    label = 'Normal' if all(group_data['Label'] == '-') else 'Anomalous'
    result_list.append({'EventIds': event_ids, 'Label': label})

result_df = pd.DataFrame(result_list)

result_df.to_csv('E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\group\proj\Thunderbird_sequence.csv', index=False)